# Pulling and Pre-Processing Listing Data   

## Import required libraries

In [49]:
from DomainAnalysis.mongo import get_all_listings, connect_to_mongo_db, connect_to_domain_listings
import pandas as pd

## Pull data from MongoDB

In [50]:
client = connect_to_mongo_db()
coll = connect_to_domain_listings(client)
listings_raw = get_all_listings(coll)

In [51]:
listings = pd.DataFrame(listings_raw)
listings.head()

,_id,listing_id,created,dateListed,dateUpdated,saleMethod,saleMode,displayPrice,minimumPrice,maximumPrice,...,house_bedrooms,house_carspaces,house_description,house_headline,house_isNewDevelopment,house_propertyType,house_landAreaSqm,agent_advertiserType,agent_advertiserId,location_unitNumber
0,6155208cfd8729c049474af5,2017278297,2021-09-30T02:26:12.895734,2021-09-24T06:43:18Z,2021-09-27T22:29:18.3Z,auction,buy,"$950,000 - $1,050,000",950000.0,1050000.0,...,3,2,OPEN FOR PRIVATE INSPECTION SAT 2ND OCTOBER - ...,"Enjoy, Invest, Best Of Both!",False,house,NaN,agency,5600,NaN
1,61552285fd8729c049474af6,2017273091,2021-09-30T02:35:31.331938,2021-09-22T09:24:07Z,2021-09-22T09:24:07.58Z,privateTreaty,buy,"$580,000",580000.0,580000.0,...,2,1,"Live with cafes at the door, a panoramic view ...","Step Onto the Train, Step Up to the Roof",False,apartmentUnitFlat,NaN,agency,3937,NaN
2,615522affd8729c049474af7,2017272108,2021-09-30T02:36:22.026958,2021-09-22T05:44:10Z,2021-09-27T07:59:22.48Z,privateTreaty,buy,"$1,350,000 - $1,400,000",1350000.0,1400000.0,...,3,2,INSPECTIONS STRICTLY BY PRIVATE APPOINTMENT ON...,Rare Family Find on the Green,False,house,533.0,agency,4316,NaN
3,61552e7fa61be1df526af669,2017287671,2021-09-30T03:26:54.873732,2021-09-29T05:46:33Z,2021-09-29T06:09:44.577Z,auction,buy,"$550,000 - $590000",550000.0,590000.0,...,2,1,INSPECTIONS BY APPOINTMENT - CONTACT AGENT\r\n...,PEACEFUL REAR POSITION,False,apartmentUnitFlat,NaN,agency,3918,NaN
4,61552e7fa61be1df526af66b,2017286716,2021-09-30T03:26:55.054060,2021-09-29T03:13:40Z,2021-09-29T06:33:11.397Z,privateTreaty,buy,"$600,000 - $650,000",600000.0,650000.0,...,2,2,OFFERS CLOSING MONDAY 18TH OCTOBER AT 1PM\r\n\...,Parkside Alfresco Entertainer!,False,townhouse,NaN,agency,3937,NaN


## Investigate data types

In [52]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597 entries, 0 to 596
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   _id                           597 non-null    object 
 1   listing_id                    597 non-null    int64  
 2   created                       597 non-null    object 
 3   dateListed                    597 non-null    object 
 4   dateUpdated                   597 non-null    object 
 5   saleMethod                    597 non-null    object 
 6   saleMode                      597 non-null    object 
 7   displayPrice                  597 non-null    object 
 8   minimumPrice                  145 non-null    float64
 9   maximumPrice                  145 non-null    float64
 10  inspectionsByAppointmentOnly  591 non-null    object 
 11  url                           597 non-null    object 
 12  statementOfInformation        565 non-null    object 
 13  locat

## Fix data types

### Date Features

In [53]:
listings.created = pd.to_datetime(listings.created)
listings.dateListed = pd.to_datetime(listings.dateListed)
listings.dateUpdated = pd.to_datetime(listings.dateUpdated)

### Factor Features

In [54]:
listings.saleMethod = listings.saleMethod.astype('category')
listings.saleMode = listings.saleMode.astype('category')
listings.location_state = listings.location_state.astype('category')
listings.location_street = listings.location_street.astype('category')
listings.location_suburb = listings.location_suburb.astype('category')
listings.location_postcode = listings.location_postcode.astype('category')
listings.house_propertyType = listings.house_propertyType.astype('category')
listings.agent_advertiserType = listings.agent_advertiserType.astype('category')
listings.agent_advertiserId = listings.agent_advertiserId.astype('category')

### Bool Features

In [55]:
listings.inspectionsByAppointmentOnly = listings.inspectionsByAppointmentOnly.astype('bool')

### Other Features
- Street number has yet to be handled. It can be used to extract unit numbers from some of these. In addition, how to handle lettered street numbers is yet to be determined. Ideally the value would be ordinal.
    - Display Address could also be used to help these values

## Fix Column Names
Some of the column names have uneeded text in them

In [56]:
listings.rename(columns={
    'inspectionsByAppointmentOnly': 'isInspectionsByAppointmentOnly',
    'location_state': 'state',
    'location_streetNumber': 'streetNumber',
    'location_street':'street',
    'location_suburb':'suburb',
    'location_postcode':'postcode',
    'location_displayAddress':'displayAddress',
    'location_latitude':'latitude',
    'location_longitude':'longitude',
    'location_unitNumber': 'unitNumber',
    'house_bathrooms':'bathrooms',
    'house_bedrooms':'bedrooms',
    'house_carspaces':'carspaces',
    'house_description':'description',
    'house_headline':'headline',
    'house_isNewDevelopment':'isNewDevelopment',
    'house_propertyType': 'propertyType',
    'house_landAreaSqm': 'landAreaSqm',
    'agent_advertiserType': 'advertiserType',
    'agent_advertiserId': 'advertiserId'
}, inplace=True)

## Extract Min & Max Prices from Display Price

Some of the display prices have a minimum and maximum value in it but that hasn't been properly extracted during the original upload. 

> The work done here will improve future uploads but doesn't help to fix existing uploads

In [57]:
# Get a subset with all the listings without a mini or max price to investigate the various display prices
listing_wo_min_max_mask = listings.minimumPrice.isna() | listings.maximumPrice.isna()
listing_wo_min_max = listings[listing_wo_min_max_mask].copy()

Okay so there's a few possible scenarios:
1. Displays only range from min to max (e.g $100,000 - $150,000)
   1. We already have a function to handle this. Possibly older listings where this function did not exist or did not work properly
2. Auction / Contact Agent
   1. These values don't display any value can the min/max price will have to be obtained through alternative measures
3. Range with other text
   1. The function doesn't currently handle other text very well so these are prime scenarios to work out how to handle now. 
4. Other text
   1. Veyr possible that there could be some other plain text that doesn't convert to min/max prices. Again alternative solutions will need to be used

Before going too far lets quickly check the count of listings that are missing a min/max value. This will be a good measure to see how we progress.

In [58]:
print(f"Count of listings without a Minimum or Maximum price: {listing_wo_min_max.displayPrice.count()}")

Count of listings without a Minimum or Maximum price: 452


### Range Display Prices

For these we can import the existing `get_minimum` and `get_maximum` functions from the wrangler module

In [59]:
from DomainAnalysis.wrangler import get_min_max_price_from_display_price
from DomainAnalysis.logger import logger
logger.disabled = True
def get_min_max(displayPrice):
    """Used to ignore errors as they're intended for the ETL process"""
    try:
        return get_min_max_price_from_display_price(displayPrice)
    except Exception as e:
        return
    
prices = listing_wo_min_max.displayPrice.apply(get_min_max).dropna()

During testing the output showed some prices that were clearly wrong. For example one was (1, 11), which I wish I could buy a house for a tenner but obviously not true. Investigating that particular listing revealed that the display price was a string of text reading: `Offers Closing Monday 11th October at 1pm`. We will add in a basic check on the minimum price to remove these but this should be looked into for the ETL script as currently this would pass and create outliers in the dataset. 

In [60]:
prices = prices[prices.apply(lambda x: x[0] > 100)]
prices

65     (1250000, 1350000)
72      (940000, 1020000)
73       (900000, 990000)
74      (990000, 1050000)
75       (680000, 740000)
              ...        
515      (900000, 950000)
517      (889000, 959000)
518      (300000, 330000)
519      (920000, 980000)
520      (930000, 970000)
Name: displayPrice, Length: 267, dtype: object

Perfect! Let's apply this to the relative records in the database.

In [61]:
for idx in prices.index:
    listings.loc[idx, 'minimumPrice'] = prices[idx][0]
    listings.loc[idx, 'maximumPrice'] = prices[idx][0]

Make sure we made some improvements

In [62]:
listing_wo_min_max_mask = listings.minimumPrice.isna() | listings.maximumPrice.isna()
listing_wo_min_max = listings[listing_wo_min_max_mask].copy()
print(f"Count of listings without a Minimum or Maximum price: {listing_wo_min_max.displayPrice.count()}")

Count of listings without a Minimum or Maximum price: 185


Noice! Heaps better. Okay let's tackle the trickier ones. It's going to be a bit hard to locate them programatically, so I think the best way will be to work backwards by printing all the faulty display prices and find patterns that way. 

> If running this notebook at a later time with substantially more listings in the database it might be better to not run this next code block and hope I did a good job with the patterns :) Or add to them whatevs..

In [63]:
import re
for idx, displayPrice in listing_wo_min_max.displayPrice.iteritems():
    # Contact agent will never have a value neither if it just reads Auction
    if 'contact agent' not in displayPrice.lower() or displayPrice.lower() != 'auction':
        print(f"Index: {idx}, Display Price: {displayPrice}")

Index: 62, Display Price: CLOSING DATE SALE - Tuesday 19th Oct @ 2pm
Index: 63, Display Price: Contact Agent
Index: 64, Display Price: Offers Closing Monday 11th October at 1pm
Index: 66, Display Price: $1,200,000$1,300,000 ONLINE AUCT WED 6/10 5PM
Index: 67, Display Price: Contact Agent
Index: 68, Display Price: UNDER CONTRACT
Index: 69, Display Price: UNDER CONTRACT
Index: 70, Display Price: Online auction this Saturday at 12:00pm!
Index: 71, Display Price: Under Offer
Index: 79, Display Price: UNDER OFFER
Index: 80, Display Price: Forthcoming Auction
Index: 83, Display Price: DIGITAL ONLINE AUCTION THIS SATURDAY AT 11AM
Index: 84, Display Price: $1,065,000
Index: 85, Display Price: Contact Agent
Index: 86, Display Price: Contact Agent
Index: 89, Display Price: $395,000
Index: 90, Display Price: Contact Agent.
Index: 91, Display Price: Contact agent
Index: 92, Display Price: Contact Agent
Index: 93, Display Price: Sale | $1,600,000
Index: 94, Display Price: Auction Sat 16th Oct at 11

Through this exercise another easy value was identified. A single price! 

We can also remove any `contact agent` values to simplify the list further

Other Findings were exactly or went in the format of:
- $1,200,000$1,300,000 ONLINE AUCT WED 6/10 5PM
- Sale | $1,600,000
- Forthcoming Auction $960,000  $1,050,000
- Auction Sat 30th Oct 12:30pm - $900,000 - $960,000
- $899,000 – $969,000 -> Didn't work previously apparently
- $725,000     Private inspections welcome
- $3,100,000   Private inspections welcome
- [word] - $870,000 - $890,000
- [word] [word] - $969,000 - $999,000
- 1.3M-1.4M
- AUCTION - EPR $1,475,000 - $1,525,000
- AUCTION - EPR $950,000 - $975,000 EACH
- Quoting $1,750,000
- EOI - EPR $2,700,000 - $2,900,000

In most circumstances the values are written in a _"readable"_ with the dollar sign and integers in groups of three. I think it will be best to use regex to match for the two values as this will cover a lot of these instances. 

We'll work that out first then revisit other values

Okay I've got the regex to extract the two prices ([see here for testing](https://regex101.com/r/1VHp80/3)). 

Basically it matches a price then anything in between followed by another price. This regex assumes that the $XXX,XXX... format is being used but as can be seen above that is most of the cases :) 

```regex
(?:(\$(?:\d{1,3},)+\d{1,3}).*(\$(?:\d{1,3},)+\d{1,3}))
```

In [71]:
prices = listing_wo_min_max.displayPrice.apply(lambda x: re.search(r'.*(\$(?:\d{1,3},)+\d{1,3}).*(\$(?:\d{1,3},)+\d{1,3})', x)).dropna()
clean_price = lambda x: x.replace("$","").replace(",","")
for idx in prices.index:
    listings.loc[idx, 'minimumPrice'] = clean_price(prices[idx].group(1))
    listings.loc[idx, 'maximumPrice'] = clean_price(prices[idx].group(2))

In [72]:
prices = listing_wo_min_max.displayPrice.apply(lambda x: re.search(r'.*(\$(?:\d{1,3},)+\d{1,3}).*(\$(?:\d{1,3},)+\d{1,3})', x)).dropna()
prices = listing_wo_min_max.displayPrice.apply(lambda x: re.search(r'.*(\$(?:\d{1,3},)+\d{1,3})', x)).dropna()
prices[93].group(1)

'$1,600,000'

Not let's handle the single values as well

In [73]:
prices = listing_wo_min_max.displayPrice.apply(lambda x: re.search(r'.*(\$(?:\d{1,3},)+\d{1,3})', x)).dropna()
for idx in prices.index:
    listings.loc[idx, 'minimumPrice'] = clean_price(prices[idx].group(1))
    listings.loc[idx, 'maximumPrice'] = clean_price(prices[idx].group(1))

In [74]:
listing_wo_min_max_mask = listings.minimumPrice.isna() | listings.maximumPrice.isna()
listing_wo_min_max = listings[listing_wo_min_max_mask].copy()
print(f"Count of listings without a Minimum or Maximum price: {listing_wo_min_max.displayPrice.count()}")

Count of listings without a Minimum or Maximum price: 130


Awesome down to 135. We've fixed 298 records. That's a pretty good effort! 

Let's do a manual check over the remaining prices and see if we can visually see another other prices still that are extractable. 

In [75]:
for idx, displayPrice in listing_wo_min_max.displayPrice.iteritems():
    # Contact agent will never have a value neither if it just reads Auction
    if 'contact agent' not in displayPrice.lower() or displayPrice.lower() != 'auction':
        print(f"Index: {idx}, Display Price: {displayPrice}")

Index: 62, Display Price: CLOSING DATE SALE - Tuesday 19th Oct @ 2pm
Index: 63, Display Price: Contact Agent
Index: 64, Display Price: Offers Closing Monday 11th October at 1pm
Index: 67, Display Price: Contact Agent
Index: 68, Display Price: UNDER CONTRACT
Index: 69, Display Price: UNDER CONTRACT
Index: 70, Display Price: Online auction this Saturday at 12:00pm!
Index: 71, Display Price: Under Offer
Index: 79, Display Price: UNDER OFFER
Index: 80, Display Price: Forthcoming Auction
Index: 83, Display Price: DIGITAL ONLINE AUCTION THIS SATURDAY AT 11AM
Index: 85, Display Price: Contact Agent
Index: 86, Display Price: Contact Agent
Index: 90, Display Price: Contact Agent.
Index: 91, Display Price: Contact agent
Index: 92, Display Price: Contact Agent
Index: 94, Display Price: Auction Sat 16th Oct at 11 AM
Index: 97, Display Price: CONTACT AGENT
Index: 100, Display Price: Forthcoming Auction
Index: 101, Display Price: Contact Agent
Index: 122, Display Price: Contact Agent
Index: 124, Dis

Okay so the single run through missed some:
- Sale | $1,600,000
- Current Offer $550,000
- Quoting $1,750,000

It's possible the regex used above didn't like it with the text above. I just copied it and pasted it from my double one and didn't test thoroughly. My tests on regex101 didn't quite carry over for the Python implementation. Adding `.*` to the front fixed it up. _This has been edited above._ 

We've also got to handle these instances:
- 1.3M-1.4M

In [76]:
def fix_short_price(displayPrice):
    """Takes a display price that has a shortened format (E.G: 2.4M - 2.6M) and extracts the minimum and maximum value as a regular value (2400000)

    Args:
        displayPrice (str): displayPrice of the listing

    Returns:
        tuple: minimumPrice in the first element and the maximumPrice is in the second
    """
    pattern = r'.*((?:\d.\d{1,2}M)|(?:\d{1,3}K)).*((?:\d.\d{1,2}M)|(?:\d{1,3}K))'
    prices = re.match(pattern, displayPrice)
    
    fix_millions = lambda displayPrice: float(displayPrice.lower().replace("m", "")) * 1000000
    fix_thousands = lambda displayPrice: float(displayPrice.lower().replace("k", "")) * 1000
    is_millions = lambda displayPrice: True if 'm' in displayPrice.lower() else False
    
    return (
        fix_millions(prices[1]) if is_millions else fix_thousands(prices[1]),
        fix_millions(prices[2]) if is_millions else fix_thousands(prices[2])
    )
    
# Filter list for the records that need this transformation
pattern = r'.*((?:\d.\d{1,2}M)|(?:\d{1,3}K)).*((?:\d.\d{1,2}M)|(?:\d{1,3}K))'
prices_mask = listing_wo_min_max.displayPrice.apply(lambda x: re.match(pattern, x)).notna()
# Apply transforamtion
prices = listing_wo_min_max[prices_mask].displayPrice.apply(fix_short_price)
for idx in prices.index:
    listings.loc[idx, 'minimumPrice'] = prices[idx][0]
    listings.loc[idx, 'maximumPrice'] = prices[idx][1]

In [77]:
listing_wo_min_max_mask = listings.minimumPrice.isna() | listings.maximumPrice.isna()
listing_wo_min_max = listings[listing_wo_min_max_mask].copy()
print(f"Count of listings without a Minimum or Maximum price: {listing_wo_min_max.displayPrice.count()}")

Count of listings without a Minimum or Maximum price: 127


Alright last visual inspection

In [78]:
for idx, displayPrice in listing_wo_min_max.displayPrice.iteritems():
    # Contact agent will never have a value neither if it just reads Auction
    if 'contact agent' not in displayPrice.lower() or displayPrice.lower() != 'auction':
        print(f"Index: {idx}, Display Price: {displayPrice}")

Index: 62, Display Price: CLOSING DATE SALE - Tuesday 19th Oct @ 2pm
Index: 63, Display Price: Contact Agent
Index: 64, Display Price: Offers Closing Monday 11th October at 1pm
Index: 67, Display Price: Contact Agent
Index: 68, Display Price: UNDER CONTRACT
Index: 69, Display Price: UNDER CONTRACT
Index: 70, Display Price: Online auction this Saturday at 12:00pm!
Index: 71, Display Price: Under Offer
Index: 79, Display Price: UNDER OFFER
Index: 80, Display Price: Forthcoming Auction
Index: 83, Display Price: DIGITAL ONLINE AUCTION THIS SATURDAY AT 11AM
Index: 85, Display Price: Contact Agent
Index: 86, Display Price: Contact Agent
Index: 90, Display Price: Contact Agent.
Index: 91, Display Price: Contact agent
Index: 92, Display Price: Contact Agent
Index: 94, Display Price: Auction Sat 16th Oct at 11 AM
Index: 97, Display Price: CONTACT AGENT
Index: 100, Display Price: Forthcoming Auction
Index: 101, Display Price: Contact Agent
Index: 122, Display Price: Contact Agent
Index: 124, Dis

Looking good. I expect we will need to look in the statement of information to extract the price from the remaining records, however, this will require further programming and interaction with a PDF document that we don't currently have locally. 

We just need to fix up the data types of both fields and we're good to go

In [96]:
# Need to convert to float then `Int64` to retain the Null values
listings.minimumPrice = listings.minimumPrice.astype(float).astype('Int64')
listings.maximumPrice = listings.maximumPrice.astype(float).astype('Int64')

In [100]:
listings.to_csv('output/half_processed.csv')